# Prototyping

Creates a neural network which evaluates a time series and produce a set of predicted values for the time series
Predicted values may be used in a policy to make a trade. This policy may be modeled by simple multiple regression or a neural network.

## Data
Test data is taken as most recent to avoid lookahead bias. Train data is split into a validation and training set during fitting.


## TODO
- Convert feature percentages to stdev
- Adding VIX as a signal
- Adding High/Low as signals
- Multiple securities/ aggregate samples
- Policy network
- Regularization (l2)
- Dilated convolution

In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import layers

dataset_path = keras.utils.get_file("SPY.csv", "http://localhost:8000/data/daily/SPY.csv")

column_names = ['Date','Open','High','Low','Close','Adj Close','Volume'] 
raw_dataset = pd.read_csv(dataset_path, 
#                           names=column_names, 
                          dtype={'Close': np.float64,'Open': np.float64,'High': np.float64,'Adj Close': np.float64, 'Volume': np.float64},
                          header=0,
                          na_values = "?", 
                          comment='\t',
                          sep=",",
                          skipinitialspace=True)


dataset = raw_dataset.copy()
dataset = dataset.sort_values(by=['Date'],ascending=False)

dataset_stats = dataset.describe()
dataset_stats = dataset_stats.transpose()
NUM_INPUT_NEURONS = 64
NUM_OUTPUT_NEURONS = 3
NUM_SAMPLES = len(dataset)
NUM_TEST_SAMPLES = 3 * NUM_SAMPLES / 10
#NUM_TEST_SAMPLES = 1000
print NUM_SAMPLES
# Create features (only close price for now)
def convert_to_percentage(old, new):
    return (old - new) / old


def convert_labels_to_category(labels): 
    # Simplification - If positive return, 1, else 0
    # return map(lambda arr: 1 if arr[0] > 1 else 0, labels)
    # rounding simpliciation, 10th of percentage
    return map(lambda arr: map(lambda val: round(val,2),arr), labels)

def convert_to_train(raw_dataset):
    dataset = raw_dataset.copy()
    features = []
    labels = []
    for i in range(5, len(dataset) - NUM_INPUT_NEURONS):

        feature_dataset = dataset[i:i+NUM_INPUT_NEURONS].copy()
        latest_close = feature_dataset['Close'].iloc[0]
        
        features.append(
            feature_dataset['Close']
                .map(lambda current: convert_to_percentage(latest_close, current))
                .tolist()
        )
        labels.append([
            dataset['Close'].iloc[i-1] / latest_close, # 1 day trade
            dataset['Close'].iloc[i-3] / latest_close, # 3 day trade
            dataset['Close'].iloc[i-5] / latest_close, # 5 day trade
        ])
        
    # Without converting labels the precision is hard to determine accuracy. 
    # Rather than crude 0/1, maybe this can be more sophisticated
    labels = convert_labels_to_category(labels)
    
    return [features,labels]
converted_feature_set = convert_to_train(dataset)
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(converted_feature_set[0][0])]),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dense(16, activation=tf.nn.relu),
    layers.Dense(NUM_OUTPUT_NEURONS)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer='sgd',
                metrics=[
                    'mae',
                #    'accuracy'
                ])
  return model

tensorboard = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

model = build_model()
model.summary()



6445
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 51        
Total params: 6,819
Trainable params: 6,819
Non-trainable params: 0
_________________________________________________________________


In [13]:
#len(converted_feature_set[0][0])
print NUM_TEST_SAMPLES
train_data = np.array(converted_feature_set[0][NUM_TEST_SAMPLES:])
train_labels = np.array(converted_feature_set[1][NUM_TEST_SAMPLES:])

test_data = np.array(converted_feature_set[0][:NUM_TEST_SAMPLES])
test_labels = np.array(converted_feature_set[1][:NUM_TEST_SAMPLES])


1933


In [14]:
print train_data
print train_labels

[[ 0.         -0.0041834   0.00160905 ...  0.08753021  0.08302494
   0.08117461]
 [ 0.          0.00576832  0.00120175 ...  0.08684503  0.08500241
   0.09878226]
 [ 0.         -0.00459307 -0.00370669 ...  0.07969379  0.09355359
   0.08904106]
 ...
 [ 0.          0.00280309  0.00630582 ... -0.00840926 -0.00911115
  -0.00490652]
 [ 0.          0.00351258  0.00983838 ... -0.01194773 -0.00773128
   0.00281097]
 [ 0.          0.0063481   0.00775853 ... -0.0112835  -0.00070409
   0.00141044]]
[[1.   1.01 1.01]
 [1.   1.   1.01]
 [1.01 1.   1.01]
 ...
 [1.   1.   1.  ]
 [1.   1.   1.  ]
 [1.   1.   1.  ]]


In [15]:

history = model.fit(
    train_data, train_labels,
    epochs=60, validation_split = 0.2, verbose=1,
    callbacks=[tensorboard]
)

Train on 3554 samples, validate on 889 samples
Epoch 1/60
3554/3554 [==============================] - 0s 87us/step - loss: 0.4572 - mean_absolute_error: 0.6465 - val_loss: 0.1163 - val_mean_absolute_error: 0.3369
Epoch 2/60
3554/3554 [==============================] - 0s 43us/step - loss: 0.0368 - mean_absolute_error: 0.1663 - val_loss: 0.0037 - val_mean_absolute_error: 0.0516
Epoch 3/60
3554/3554 [==============================] - 0s 52us/step - loss: 0.0018 - mean_absolute_error: 0.0321 - val_loss: 3.3599e-04 - val_mean_absolute_error: 0.0152
Epoch 4/60
3554/3554 [==============================] - 0s 44us/step - loss: 9.1221e-04 - mean_absolute_error: 0.0210 - val_loss: 2.2185e-04 - val_mean_absolute_error: 0.0121
Epoch 5/60
3554/3554 [==============================] - 0s 41us/step - loss: 8.5830e-04 - mean_absolute_error: 0.0200 - val_loss: 2.0097e-04 - val_mean_absolute_error: 0.0113
Epoch 6/60
3554/3554 [==============================] - 0s 42us/step - loss: 8.3594e-04 - mean_abs

3554/3554 [==============================] - 0s 47us/step - loss: 6.1631e-04 - mean_absolute_error: 0.0174 - val_loss: 1.6307e-04 - val_mean_absolute_error: 0.0099
Epoch 48/60
3554/3554 [==============================] - 0s 43us/step - loss: 6.1445e-04 - mean_absolute_error: 0.0174 - val_loss: 1.6276e-04 - val_mean_absolute_error: 0.0099
Epoch 49/60
3554/3554 [==============================] - 0s 43us/step - loss: 6.1265e-04 - mean_absolute_error: 0.0174 - val_loss: 1.6248e-04 - val_mean_absolute_error: 0.0098
Epoch 50/60
3554/3554 [==============================] - 0s 42us/step - loss: 6.1094e-04 - mean_absolute_error: 0.0173 - val_loss: 1.6158e-04 - val_mean_absolute_error: 0.0098
Epoch 51/60
3554/3554 [==============================] - 0s 39us/step - loss: 6.0907e-04 - mean_absolute_error: 0.0173 - val_loss: 1.6287e-04 - val_mean_absolute_error: 0.0099
Epoch 52/60
3554/3554 [==============================] - 0s 42us/step - loss: 6.0750e-04 - mean_absolute_error: 0.0173 - val_loss: 1

In [16]:
outputs = model.predict(test_data)


In [17]:

# one day return
actual = map(lambda arr: arr[0],outputs)
# signal step for our policy network
signals = map(lambda arr: sum(arr) / len(arr),outputs)
# primitive policy temporarily in place of a RL policy network
trades = map(lambda signal: 1 if round(signal,2) > 1 else 0,signals)

In [18]:
df = pd.DataFrame({
    'signal':signals,
#    'actual':test_labels,
    'actual': actual,
    'trade':trades,
})

In [19]:
df['entry_success'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 1) else 0,axis=1)
df['entry_failure'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 1) else 0,axis=1)
df['avoid_success'] = df.apply (lambda row: 1 if (row['actual'] < 1.00 and row['trade'] == 0) else 0,axis=1)
df['avoid_failure'] = df.apply (lambda row: 1 if (row['actual'] > 1.00 and row['trade'] == 0) else 0,axis=1)


In [20]:
# primitive policy - replace with a policy network which maximizes reward
def label_success (row):
    return 0 if (row['entry_failure'] == 1 or row['entry_failure'] == 1) else 1

success = df.apply (lambda row: label_success (row),axis=1)
df['success'] = success;
df

,actual,signal,trade,entry_success,entry_failure,avoid_success,avoid_failure,success
0,1.007966,1.008044,1,1,0,0,0,1
1,1.006003,1.006680,1,1,0,0,0,1
2,1.004104,1.005510,1,1,0,0,0,1
3,1.006690,1.008047,1,1,0,0,0,1
4,1.006368,1.006969,1,1,0,0,0,1
5,1.005700,1.006164,1,1,0,0,0,1
6,1.006684,1.005058,1,1,0,0,0,1
7,1.005771,1.002381,0,0,0,0,1,1
8,1.004072,1.002482,0,0,0,0,1,1
9,1.006717,1.004700,0,0,0,0,1,1


In [21]:
print '\nNon-loss events'
print sum(df['success'])
print sum(df['success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nLose trades'
print sum(df['entry_failure'])
print sum(df['entry_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nWin trades'
print sum(df['entry_success'])
print sum(df['entry_success']) / (NUM_TEST_SAMPLES * 1.00)

print '\nMissed opportunities'
print sum(df['avoid_failure'])
print sum(df['avoid_failure']) / (NUM_TEST_SAMPLES * 1.00)

print '\nBullets dodged'
print sum(df['avoid_success'])
print sum(df['avoid_success']) / (NUM_TEST_SAMPLES * 1.00)


Non-loss events
1914
0.990170719089

Lose trades
19
0.0098292809105

Win trades
439
0.22710812209

Missed opportunities
987
0.510605276772

Bullets dodged
488
0.252457320228
